In [1]:
dbutils.library.installPyPI('nltk')

import nltk
nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data] Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data] Unzipping tokenizers/punkt.zip.
 Out[ 1 ]: True

In [2]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from collections import Counter
import pandas as pd
import time

def preprocess(string):
    words = word_tokenize(string.lower())
    return words

In [3]:
articles = pd.read_csv("/dbfs/FileStore/tables/articles1.csv", usecols=[1,2,9])
articles2 = pd.read_csv("/dbfs/FileStore/tables/articles2.csv", usecols=[1,2,9])
articles3 = pd.read_csv("/dbfs/FileStore/tables/articles3.csv", usecols=[1,2,9])

In [4]:
articlesFinal = pd.concat([articles, articles2, articles3])
articlesFinal = pd.concat([articles, articles2, articles3])

In [5]:
articlesFinal['content']= articlesFinal['content'].str.lower()
articlesFinal['title'] = articlesFinal['title'].str.lower()

In [6]:
articlesFinal.to_csv('/dbfs/FileStore/tables/results.csv',index=False,header=True)

In [7]:
# Asi se lee 
articles4 = pd.read_csv("/dbfs/FileStore/tables/results.csv")

In [8]:
invertedIndex = dict()
titles = dict()

#Contador de palabras por articulo que crea un diccionario
def word_count(line,id):    
    counts = dict()
    words = line.split(' ')
    for word in words: 
      if word in invertedIndex and id in invertedIndex[word]:
        invertedIndex[word][id] += 1
      elif word in invertedIndex and id not in invertedIndex[word]:
        invertedIndex[word].update({id:1})
      else:
        invertedIndex[word] = {id:1}

#Diccionario de títulos (opcional)
def titleDict(id,title):
   titles[id] = title
    
#Se pasa cada articulo para hacer el indice
for index, row in articlesFinal.iterrows():
    word_count(row["content"],row["id"])
    titleDict(row["id"],row["title"])
    

In [9]:
import operator

#Método para buscar en el indice invertido
def searchWord(word):
  if(word.lower() not in invertedIndex):
    print("Word not found")
    return
  articles = list(invertedIndex[word.lower()].items())
  articles.sort(key = lambda x: x[1], reverse = True) 
  
  for id,freq in articles[:5]:
    print(freq, id, titles[id])

In [10]:
searchWord("house")

In [11]:
dbutils.library.installPyPI('gensim')
from gensim.models.doc2vec import Doc2Vec, LabeledSentence

In [12]:
sentences = articles4.content
ids = articles4.id
titles = articles4.title

tagged_data = [LabeledSentence(preprocess(str(titles[i]) + ' ' + str(sentences[i])),[ids[i]]) for i in range(len(ids))]

In [13]:
import multiprocessing 

max_epochs = 1
vec_size = 100
alpha = 0.025

model = Doc2Vec(documents = tagged_data,
                vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1,
                workers = multiprocessing.cpu_count())
#model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("/dbfs/FileStore/d2v.model")
print("Model Saved")

iteration 0
Model Saved

In [14]:
model= Doc2Vec.load("/dbfs/FileStore/d2v.model")

In [15]:
test = preprocess(articles4.title[35764] + ' ' + articles4.content[35764])
v1 = model.infer_vector(test)

In [16]:
model.docvecs.most_similar([v1])

Out[ 82 ]: 
[(55400, 0.9111143350601196),
 (58458, 0.7080305218696594),
 (56202, 0.6731009483337402),
 (170157, 0.6715680360794067),
 (201306, 0.6699118614196777),
 (20663, 0.6671432256698608),
 (58672, 0.6423705816268921),
 (58300, 0.6375399827957153),
 (41637, 0.6339700222015381),
 (204315, 0.6111174821853638)]

In [17]:
print(articles4.iloc[35764])
print(articles4[articles4.id == 58458])

id 55400
title cnn’s reality check team vets trump’s claims
content washington (cnn) donald trump on wednesday gav...
Name: 35764, dtype: object
 id title \
38319 58458 fact-checking the 2nd debate 

 content 
38319 (cnn) hillary clinton and donald trump met su...